In [1]:
import json
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from utils import *
from natsort import natsorted
from torch.utils.data import Dataset
from Get_model_and_data import *
import torch.nn.functional as F

c:\Users\PC\anaconda3\envs\seg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.a = nn.Parameter(torch.tensor(0.5))
        self.b = nn.Parameter(torch.tensor(0.5))

    def forward(self, x, y):
        return self.a * x + self.b * y

In [46]:

device = "cpu"
train_mask_dir = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/train"
array_folder_local = f"C:/Users/PC/Desktop/Segmentation/out/2024_May_18-11_51_34/pred_probs_caglar_he_train"
array_folder_global = f"C:/Users/PC/Desktop/Segmentation/out/2024_May_18-11_58_31/pred_probs_caglar_he_train"

dataset = Global_Local_Dataset(array_folder_local, array_folder_global, train_mask_dir)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)

model = LinearModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.5**(epoch // 15))




In [37]:
def get_max_values_from_arrays(folder_path):
    max_values = []

    # List all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):  
            file_path = os.path.join(folder_path, filename)
            
            # Load the array
            array = np.load(file_path)
            
           
            max_value = np.max(array)
            
          
            max_values.append(max_value)
    
    return max_values

In [38]:
a = get_max_values_from_arrays(f"C:/Users/PC/Desktop/Segmentation/out/2024_May_18-11_58_31/pred_probs_caglar_he_train")

In [ ]:
a

In [ ]:
for batch_idx, (local_global_features, masks) in enumerate(train_loader):
        print(f"Batch {batch_idx+1}")
        print(f"Local Features Shape: {local_global_features.shape}")
        print(f"Masks Shape: {masks.shape}")
        print(torch.max(local_global_features[0][1]),torch.max(local_global_features[0][0]))
        print(local_global_features)
        

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for merged_array, gt_mask in train_loader:
        merged_array, gt_mask = merged_array.to(device), gt_mask.to(device)
        optimizer.zero_grad()
        output = model(merged_array[0].float(),merged_array[1].float())
        loss = criterion(output, gt_mask.float())
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Learning Rate: {scheduler.get_last_lr()[0]}")
    print(f"Epoch {epoch+1}, Coefficients: a = {model.a.item()}, b = {model.b.item()}")

In [ ]:
def predict_and_save_results(global_dir, local_dir, model_path, device, saved_prob_dir,saved_mask_dir,gt_folder_test):
    if not os.path.exists(saved_prob_dir):
        os.makedirs(saved_prob_dir, exist_ok=True)
    if not os.path.exists(saved_mask_dir):
        os.makedirs(saved_mask_dir, exist_ok=True)

    model = LinearModel().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    array_folder_local = f"{local_dir}/pred_probs_caglar_he"
    array_folder_global = f"{global_dir}/pred_probs_caglar_he"
    

    dataset = Global_Local_Dataset(array_folder_local, array_folder_global, gt_folder_test)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

    with torch.no_grad():
        for idx, (merged_array, gt_mask) in enumerate(data_loader):
            merged_array = merged_array.to(device)
            output = model(merged_array.float())
            output_np = output.cpu().numpy().squeeze()

            save_path = os.path.join(saved_prob_dir, f"prediction_{idx}.npy")
            np.save(save_path, output_np)
        

            png_save_path = os.path.join(saved_mask_dir, f"prediction_{idx}.png")
            output_bool = (output_np > 0.5).astype(np.uint8)
            output_img = (output_bool * 255).astype(np.uint8)
            cv2.imwrite(png_save_path, output_img)
        print(f"Prediction masks are  saved")
        print(f"Prediction images are saved")

# Set device and directories
device = "cuda" if torch.cuda.is_available() else "cpu"
train_mask_directory = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/train"
global_dir = "C:/Users/PC/Desktop/Segmentation/out/2024_May_18-11_58_31/"
local_dir = "C:/Users/PC/Desktop/Segmentation/out/2024_May_18-11_51_34/"
save_model_path = "Final_model_results/"
saved_prob_dir = f"{save_model_path}prob_arrays/"
saved_mask_dir = f"{save_model_path}mask_arrays/"
gt_folder_test = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/test" 
gt_mask = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/train/" 
# Train the model and save it
global_local_prob_model(train_mask_directory,global_dir, local_dir, device, save_model_path)

# Load the model, make predictions, and save them
predict_and_save_results(global_dir, local_dir, f"{save_model_path}best_model.pth", device, saved_prob_dir,saved_mask_dir,gt_folder_test)


In [5]:
test_Array = np.load("C:/Users/PC/Desktop/Segmentation/IDRiD-Eye-Fundus-Dataset-Lesion-Segmentation/Final_model_results/prob_arrays/prediction_1.npy")
np.max(test_Array)

0.834802

In [11]:
plot_save_mismatches("C:/Users/PC/Desktop/Segmentation/IDRiD-Eye-Fundus-Dataset-Lesion-Segmentation/Final_model_results/mask_arrays",
                     gt_folder_test,
                     "C:/Users/PC/Desktop/Segmentation/IDRiD-Eye-Fundus-Dataset-Lesion-Segmentation/Final_model_results",
                     mismatched_images="mismatched_images_caglar")
logging.info("Plotting and saving mismatches completed successfully.")


Plotting mismatches: 27it [00:07,  3.68it/s]
2024-05-18 19:40:23,811 - root - INFO - Plotting and saving mismatches completed successfully.


In [3]:
auc_pr_result_paper_caglar,_,_= auc_pr_paper_calculation(pred_mask_dir="C:/Users/PC/Desktop/Segmentation/IDRiD-Eye-Fundus-Dataset-Lesion-Segmentation/Final_model_results/prob_arrays", 
                                                        test_mask_dir=gt_folder_test,
                                                        
                                                        stride=576)

In [4]:
auc_pr_result_paper_caglar

0.5903840984589883

In [ ]:
auc_pr_result_paper_caglar,_,_= auc_pr_paper_calculation(pred_mask_dir=log_dir+f"pred_probs_caglar_{dataset_conf['data']}", 
                                                                                        test_mask_dir=os.path.join(dataset_conf['test_mask_dir'],
                                                                                                                    dataset_conf['data']), 
                                                                                                                    stride=dataset_conf['stride'])
logging.info("AUC-PR calculation according to paper completed successfully.")
metrics_caglar = calculate_metrics(os.path.join(dataset_conf['test_mask_dir'],
                                                dataset_conf['data']), 
                                                log_dir+f"pred_masks_caglar_{dataset_conf['data']}")
logging.info("Metrics calculation completed successfully.")
#wandb.log(wandb_final_log(auc_pr_caglar=auc_pr_result_paper_caglar,metrics_caglar=metrics_caglar))
results = { "auc_pr":auc_pr_result_paper_caglar,"metrics":metrics_caglar}
json_file_path = os.path.join(log_dir, 'results.json')
with open(json_file_path, 'w') as json_file:
    json.dump(results, json_file, indent=4)
logging.info("Results saved successfully.")